In [1]:
import pyspark as ps
import pandas as pd
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [5]:
df_reservations = spark.read.csv('../data/reservations2018.csv', header=True)

In [8]:
df_reservations.printSchema()

root
 |-- HistoricalReservationID: string (nullable = true)
 |-- OrderNumber: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- OrgID: string (nullable = true)
 |-- CodeHierarchy: string (nullable = true)
 |-- RegionCode: string (nullable = true)
 |-- RegionDescription: string (nullable = true)
 |-- ParentLocationID: string (nullable = true)
 |-- ParentLocation: string (nullable = true)
 |-- LegacyFacilityID: string (nullable = true)
 |-- Park: string (nullable = true)
 |-- SiteType: string (nullable = true)
 |-- UseType: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- EntityType: string (nullable = true)
 |-- EntityID: string (nullable = true)
 |-- FacilityID: string (nullable = true)
 |-- FacilityZIP: string (nullable = true)
 |-- FacilityState: string (nullable = true)
 |-- FacilityLongitude: string (nullable = true)
 |-- FacilityLatitude: string (nullable = true)
 |-- CustomerZIP: string (nullable = true)
 |-- CustomerState: string (nullable

In [9]:
df_reservations.createOrReplaceTempView('reservations')

In [10]:
result = spark.sql("SELECT DISTINCT EntityType FROM reservations")
result.show()

+----------+
|EntityType|
+----------+
|      Tour|
|         0|
|      null|
|       NLD|
|       USA|
|       ERR|
|      Site|
+----------+



In [7]:
result = spark.sql("SELECT DISTINCT EntityType, Count(*) FROM reservations GROUP BY EntityType")
result.show()

AnalysisException: 'Table or view not found: reservations; line 1 pos 42'

#### Number of reservations by campsite

In [11]:
result = spark.sql("SELECT DISTINCT EntityID, Count(*) as reservation_count FROM reservations WHERE EntityType = 'Site'  GROUP BY EntityID SORT BY reservation_count DESC")
result.toPandas().to_csv('../data/campsite_reservation_count.csv', header=True, index=False)


#### Entity IDs ranked by income

In [62]:
result = spark.sql("SELECT DISTINCT EntityID, SUM(TotalPaid) as site_income FROM reservations WHERE EntityType = 'Site' GROUP BY EntityID ORDER BY site_income DESC ")
result.show()

+--------+------------------+
|EntityID|       site_income|
+--------+------------------+
|    null|        1589984.27|
|   95284|           76050.0|
|    5491|           59930.0|
|   25825|           46210.0|
|   77205|           45940.0|
|   69885|           43590.0|
|   44303|           39180.0|
|   69712|           38175.0|
|   82582|           37680.0|
|   69843|           36100.0|
|   92108|           35365.0|
|   89680|           34500.0|
|    7608|           34289.0|
|   77983|           33882.5|
|   86152|           33785.0|
|   57682| 33425.79999999998|
|   77452|           31480.0|
|   96752|           30820.0|
|  101361|29322.090000000007|
|  101360|29300.579999999984|
+--------+------------------+
only showing top 20 rows



In [19]:
result = spark.sql("SELECT SUM(TotalPaid) as total_income FROM reservations WHERE EntityType = 'Site' ")
result.show()

+-------------------+
|       total_income|
+-------------------+
|1.401079146200006E8|
+-------------------+



In [24]:
result = spark.sql("SELECT DISTINCT EntityID, CustomerZIP FROM reservations WHERE EntityType = 'Site' GROUP BY EntityID, CustomerZIP ")
result.show()

+--------+-----------+
|EntityID|CustomerZIP|
+--------+-----------+
|   57075|      63068|
|   95012|  773991006|
|   38283|      55804|
|   57113|      80249|
|   44371| 80122-2205|
|   31298|      76052|
|   33459|      84129|
|   71911| 95660-4137|
|   69878|      93953|
|   66016|      95695|
|   71301|      92707|
|   66106|      91504|
|   71181|      92399|
|   64520|      90275|
|   64520|      90250|
|   81095|  985321607|
|   48602|      27540|
|   42927|      89523|
|   19283|      92618|
|    9340|      84067|
+--------+-----------+
only showing top 20 rows



In [53]:
result = spark.sql("SELECT EntityID, SUM(DATEDIFF( EndDate , StartDate )) as reservation_length FROM reservations WHERE EntityType = 'Site' GROUP BY EntityID ORDER BY reservation_length DESC ")
result.show()

+--------+------------------+
|EntityID|reservation_length|
+--------+------------------+
|    null|             76292|
|    7608|              2552|
|   79800|              1613|
|   79158|              1531|
|   79828|              1494|
|   79793|              1465|
|   79258|              1178|
|   79274|              1082|
|   79579|              1040|
|   79147|               959|
|   79866|               955|
|   79696|               838|
|   79118|               738|
|   79460|               725|
|   38574|               724|
|   38675|               652|
|   90112|               589|
|   19579|               567|
|   90519|               566|
|   90524|               554|
+--------+------------------+
only showing top 20 rows



#### Why are there multiple orders for the same time period per campsite?

In [71]:
result = spark.sql("SELECT OrderNumber, EntityID, StartDate, EndDate, DATEDIFF( EndDate , StartDate ) as reservation_length, NumberOfPeople  FROM reservations WHERE EntityType = 'Site' AND EntityID = '79800' ORDER BY StartDate DESC ")
result.show()

+-----------+--------+-------------------+----------+------------------+--------------+
|OrderNumber|EntityID|          StartDate|   EndDate|reservation_length|NumberOfPeople|
+-----------+--------+-------------------+----------+------------------+--------------+
| 2-42594529|   79800|2019-03-29 00:00:00|2019-03-31|                 2|             4|
| 2-42576379|   79800|2019-03-23 00:00:00|2019-03-24|                 1|             2|
| 2-42445177|   79800|2019-03-23 00:00:00|2019-03-24|                 1|             4|
| 2-42551868|   79800|2019-03-22 00:00:00|2019-03-24|                 2|             4|
| 2-42567525|   79800|2019-03-22 00:00:00|2019-03-23|                 1|             2|
| 2-42504669|   79800|2019-03-16 00:00:00|2019-03-17|                 1|             4|
| 2-42498127|   79800|2019-03-16 00:00:00|2019-03-17|                 1|             4|
| 2-42463770|   79800|2019-03-09 00:00:00|2019-03-10|                 1|             1|
| 2-42434544|   79800|2019-03-08